In [1]:
import pandas as pd
import csv
import gzip
import json

def read_pricing_data(input_filename):
    rows = []
    with gzip.open(input_filename, 'rt') as f:
        try:
            for line in f:
                row = json.loads(line.rstrip("\n"))
                #print(row)
                timestamp_ms_received = row[0]
                price = row[2]
                rows.append({'timestamp_ms_received': timestamp_ms_received, 'price': price})
        except EOFError:
            pass

    df = pd.DataFrame(rows)
    df["timestamp"] = pd.to_datetime(df["timestamp_ms_received"], unit="ms")
    df.set_index('timestamp', inplace=True)

    return df


In [2]:
def process_pricing_data(df,output_filename):
    df = df.resample('10S').std()
    result = pd.DataFrame({
        'timestamp': df.index,
        'std': df['price']
    })
    result['timestamp'] = result['timestamp'].astype(int) // 10**6
    result.to_csv(output_filename, index=False)
    return result

In [3]:
price_df = read_pricing_data('tf-binance-BTC_ETH-2022-07-21.gz')
process_pricing_data(price_df,'out.csv')

,timestamp,std
timestamp,,
2022-07-21 00:00:00,1658361600000,0.000009
2022-07-21 00:00:10,1658361610000,0.000002
2022-07-21 00:00:20,1658361620000,0.000001
2022-07-21 00:00:30,1658361630000,0.000004
2022-07-21 00:00:40,1658361640000,0.000003
...,...,...
2022-07-21 09:05:50,1658394350000,0.000002
2022-07-21 09:06:00,1658394360000,0.000008
2022-07-21 09:06:10,1658394370000,0.000028
